# Fall 2024

In [17]:
# The following snippet will automatically reload imported code in helpers.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import csv
from datetime import date, time, timedelta
from typing import Dict, List

from helpers import calendar_table, new_syllabus, show_calendar, Plan, meeting_days, Kind, cut, schedule, merge, serialize, generate_VL_index_page, make_VL_page
from helpers import JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, DEC, MON, WED, FRI, TUES, THUR


ModuleNotFoundError: No module named 'tabulate'

## Key Dates

In [20]:
YEAR = 2024

FDOC = date(YEAR, AUG, 19)

GRADESCOPE = "https://www.gradescope.com/"
GSCOPE_DICT = {"": GRADESCOPE}

PRACTICE_PROBLEMS = "/resources/practice/practice-problems.html"

# # SPRING HOLIDAYS
# HOLIDAYS = {
#     date(YEAR, JAN, 15),     # MLK JR Day
#     date(YEAR, FEB, 12),     # Well-being Day
#     date(YEAR, FEB, 13),    # Well-being Day
#     date(YEAR, MAR, 28),    # Well-being Day
#     date(YEAR, MAR, 11),    # Spring Break
#     date(YEAR, MAR, 12),    # Spring Break
#     date(YEAR, MAR, 13),    # Spring Break
#     date(YEAR, MAR, 14),    # Spring Break
#     date(YEAR, MAR, 15),    # Spring Break
#     date(YEAR, MAR, 29),    # Univ. Holiday
    
# }


# FALL HOLIDAYS 
HOLIDAYS = {
    date(YEAR, SEP, 2),     # Labor Day
    date(YEAR, OCT, 11),    # University Day 
    date(YEAR, SEP, 3),     # Well-being Day
    date(YEAR, SEP, 23),    # Well-being Day
    date(YEAR, OCT, 17),    # Fall Break
    date(YEAR, OCT, 18),    # Fall Break
    date(YEAR, NOV, 27),    # Thanksgiving Break
    date(YEAR, NOV, 28),    # Thanksgiving Break
    date(YEAR, NOV, 29),    # Thanksgiving Break
}

DUKE_GAMES = {}

DROP_PERIOD_ENDS = date(YEAR, AUG, 30)
WITHDRAW_PERIOD_ENDS = date(YEAR, OCT, 11)

LDOC = date(YEAR, DEC, 4)

MEETING_DAYS = { MON, WED, FRI }

START_TIME = time(9, 30, 0)
END_TIME = time(10, 45, 0)

FINALS = [date(YEAR, DEC, 7)]
FINALS_TIMES = [time(8, 0, 0)]

TIMESPAN = (START_TIME, END_TIME)

## Skeleton

Let's start from the skeleton of a semester with holidays, important semester milestones, Duke games, and lecture days pencilled in.

In [ ]:
syllabus: Dict[date, List[Plan]] = new_syllabus(FDOC, LDOC)

for holiday in HOLIDAYS:
    syllabus[holiday].append(Plan("University Holiday", Kind.NA, 0, holiday))

for duke_game in DUKE_GAMES:
    syllabus[duke_game].append(Plan("Duke Game", Kind.NA, 0, duke_game))

syllabus[DROP_PERIOD_ENDS].append(Plan("Drop Period Ends", Kind.NA, 0, DROP_PERIOD_ENDS)) 
syllabus[WITHDRAW_PERIOD_ENDS].append(Plan("Widthdraw Period Ends", Kind.NA, 0, WITHDRAW_PERIOD_ENDS))

MEETINGS = meeting_days(FDOC, LDOC, MEETING_DAYS, HOLIDAYS)
LECTURES = []
for date_idx in range(len(MEETINGS)):
        day = MEETINGS[date_idx]
        lecture = Plan(f"Day {date_idx:02}", Kind.CL, date_idx, day)
        LECTURES.append(lecture)
        syllabus[day].append(lecture)

FN = []
for final_idx in range(len(FINALS)):
    final_time = FINALS_TIMES[final_idx].strftime("%-I:%M%p")
    FN.append(schedule(f"Final @ {final_time}", syllabus, Kind.FN, 0, FINALS[final_idx]))

table = calendar_table(syllabus)
with open('schedule.csv', "w") as csvfile:
    for row in table:
        print(",".join(row))
        csvfile.write(",".join(row) + "\n")


## Quizzes

In [22]:
# Update the following tuple to reflect meeting days quizzes are held on.
QZ_MEETING_DAYS = (7, 13, 25, 31, 37)
#QZ_MEETING_DAYS = (5, 11, 17, 23)

quizzes = cut(syllabus, FDOC, LDOC)

QZ: Plan = list()


for i in range(len(QZ_MEETING_DAYS)):
    quiz_date = MEETINGS[QZ_MEETING_DAYS[i]]
    QZ.append(schedule(f"Quiz {i}", quizzes, Kind.QZ, i, quiz_date, quiz_date))
#show_calendar(quizzes)

In [23]:

#TODO: For Sp24, change back to end first unit at QZ[0]
UNIT_DATES = [
    { "start": FDOC, "end": QZ[1].date },
    { "start": QZ[0].date + timedelta(days=1), "end": QZ[1].date },
    { "start": QZ[1].date + timedelta(days=1), "end": QZ[2].date },
    { "start": QZ[2].date + timedelta(days=1), "end": QZ[3].date },
    { "start": QZ[3].date + timedelta(days=1), "end": FN[-1].date }
]

unit: list[dict[date, list[Plan]]] = []
for i in range(len(UNIT_DATES)):
    unit_dates = UNIT_DATES[i]
    unit.append(cut(syllabus, UNIT_DATES[i]["start"], UNIT_DATES[i]["end"]))


In [24]:
# Collections for various components of the course
CL: list[Plan] = []
LS: list[Plan] = []
EX: list[Plan] = []
CQ: list[Plan] = []
RD: list[Plan] = []
VL: list[Plan] = []

# These indices are kind of a strange idea at first glance. The problem we are hoping to solve is
# keeping track of index numbers (to count up indices of each course component), but we'd like 
# to be able to reexecute unit cells below. So what we do is keep an index for each of the units.
# When subsequent units begin, they start from the count of the previous range (and 0 in the first).
# This way, you can reexecute cells below multiple times without having to reinitialize these indices.
# This is related to the trick of cutting / pasting the lesson plans.
CL_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
LS_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
EX_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
CQ_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
RD_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]
VL_idxs: list[int] = [0 for _ in range(len(UNIT_DATES))]

LS_due_delta: timedelta = timedelta(days=0)
VL_due_delta: timedelta = timedelta(days=1)
CQ_due_delta: timedelta = timedelta(days=0)
EX_due_delta: timedelta = timedelta(days=6)
RD_due_delta: timedelta = timedelta(days=20)

LS_VIDEOS = []
LS_links = {}

LDOC = MEETINGS[len(MEETINGS)-1]
# Auto populate LS
for idx in range(100):
    LS.append(schedule("", unit[0], Kind.LS, LS_idxs[0], LDOC))


# Functions to Create Assignments

In [25]:
def add_LS(name: str, links: dict, due_date: timedelta = LS_due_delta):
    global LS_idxs
    global LS
    global date_ctx
    global u
    global LS_VIDEOS
    global LS_links
    LS[LS_idxs[u]] = (schedule(name, unit[u], Kind.LS, LS_idxs[u], date_ctx, date_ctx + due_date))
    LS[LS_idxs[u]].links = links
    LS_idxs[u] += 1
    #LS_VIDEOS.append([name, links])
    #LS_links[name] = links
        

def add_EX(name: str, links: dict, due_date: timedelta = EX_due_delta):
    global EX_idxs
    global EX
    global date_ctx
    global u
    EX.append(schedule(name, unit[u], Kind.EX, EX_idxs[u], date_ctx, date_ctx + due_date))
    EX[EX_idxs[u]].links = links
    EX_idxs[u] += 1
    
def add_CL(name: str, links: dict):
    global CL_idxs
    global CL
    global date_ctx
    global u
    global LS_links
    CL.append(schedule(name, unit[u], Kind.CL, CL_idxs[u], date_ctx))
    CL[CL_idxs[u]].links = links
    CL_idxs[u] += 1
    LS_links[name] = links

    
def add_CQ(name: str, links: dict, due_date = timedelta(days=0)):
    global CQ_idxs
    global CQ
    global date_ctx
    global u
    CQ.append(schedule(name, unit[u], Kind.CQ, CQ_idxs[u], date_ctx, date_ctx + timedelta(days=0) + due_date))
    CQ[CQ_idxs[u]].links = links
    CQ_idxs[u] += 1
    

    

In [26]:
u = 0  # Current unit index

# Reset stateful counters and unit schedule
CL_idxs[u] = 0 # First unit starts CL from 0
CL = CL[:CL_idxs[u]] # Reset CL list - redundant in first unit, but keeping for consistency
VL_idxs[u] = 0 # First unit starts VL from 0
VL = VL[:VL_idxs[u]] # Reset VL list - redundant in first unit, but keeping for consistency
LS_idxs[u] = 0 # First unit starts LS from 0
# LS = LS[:LS_idxs[u]] # Reset LS list - redundant in first unit, but keeping for consistency
EX_idxs[u] = 0 # First unit starts EX from 0
EX = EX[:EX_idxs[u]] # Reset LS list - redundant in first unit, but keeping for consistency
CQ_idxs[u] = 0 # First unit starts CQ from 0
# CQ = CQ[:CQ_idxs[u]] # Reset LS list - redundant in first unit, but keeping for consistency
RD_idxs[u] = 0 # First unit starts CQ from 0
RD = RD[:RD_idxs[u]] # Reset RD list - redundant in first unit, but keeping for consistency
unit[u] = cut(syllabus, UNIT_DATES[u]["start"], UNIT_DATES[u]["end"])

# Date Context is the current Meeting # we're anchoring off of. We make this a variable to make
# moving around scheduled items easier.

# Week 0
date_ctx = MEETINGS[0]

add_CL("Welcome to COMP110", {"Sections 01 and 02": "/static/slides/alyssa/CL00.pdf", 
                              "Sections 03 and 04": "/static/slides/izzi/CL00Hinks.pdf"})

add_LS("Syllabus", GSCOPE_DICT)

LS[LS_idxs[u]-1].add_extension(timedelta(days=2))

date_ctx = MEETINGS[1]

add_CL(
    "Objects and Data Types",
    {
        "Sections 01 and 02": "/static/slides/alyssa/CL01.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL01Hinks.pdf",
    },
)

add_LS("VS Code, Terminal, + Running a Program", GSCOPE_DICT)

add_LS("Objects and Data Types", GSCOPE_DICT)

add_EX("Hello, world!", {"": "/exercises/hello_world.html"})

date_ctx = MEETINGS[2]

add_CL(
    "Expressions",
    {
        "Sections 01 and 02": "/static/slides/alyssa/CL02.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL02Hinks.pdf",
    },
)

add_LS("Expressions", GSCOPE_DICT)

# Week 1
date_ctx = MEETINGS[3]

add_CL(
    "Functions",
    {
        "Sections 01 and 02": "/static/slides/alyssa/CL03.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL03Hinks.pdf",
    },
)

add_LS("Introducing Functions", GSCOPE_DICT)
add_LS("Function Syntax", GSCOPE_DICT)

date_ctx = MEETINGS[4]

add_CL(
    "Practice with Functions",
    {
        "Sections 01 and 02": "/static/slides/alyssa/CL04.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL04Hinks.pdf",
    },
)

add_CQ("Writing Functions", {"": "/cqs/CQ00-functions.html"})


add_EX("Tea Party", {"": "/exercises/tea_party.html"}, timedelta(15)) # Due Sep 12

date_ctx = MEETINGS[5]

add_CL(
    "Memory Diagrams",
    {
        "Sections 01 and 02": "/static/slides/alyssa/CL05.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL05Hinks.pdf",
    },
)

add_LS("Memory Diagrams", GSCOPE_DICT)

add_CQ("Memory Diagrams", GSCOPE_DICT)

date_ctx = MEETINGS[6]

add_CL("Practice with Memory Diagrams", 
       {
        "Sections 01 and 02": "/static/slides/alyssa/CL06.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL06Hinks.pdf",
        })

date_ctx = MEETINGS[7]

QZ[0].links = {
    "What to Expect": "/resources/quiz-expectations.html",
    "Practice Problems": "/resources/practice/practice-problems.html",
    "Review Session Recording": "https://youtu.be/Og1cqDH3dKc",
}

In [27]:
date_ctx = MEETINGS[8]

add_CL("Boolean Operators and Conditionals", 
       {
        "Sections 01 and 02": "/static/slides/alyssa/CL07.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL07Hinks.pdf",
        })

add_LS("Conditionals", GSCOPE_DICT)

add_LS("Boolean Operators", GSCOPE_DICT)

date_ctx = MEETINGS[9]

add_CL(
    "Local Variables and User Input",
    {
        "Sections 01 and 02": "/static/slides/alyssa/CL08.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL08Hinks.pdf"
    },
)

add_LS("Variables and User Input", GSCOPE_DICT)

date_ctx = MEETINGS[10]


add_CQ("Practice with Conditionals, Local Variables, and User Input", {"": "/cqs/conditionals.html"})

add_EX("Chardle", {"": "/exercises/wordle-pt1.html"}, timedelta(11))  # Due date the 24th

date_ctx = MEETINGS[11]

add_CL("while Loops", {
        "Sections 01 and 02": "/static/slides/alyssa/CL09.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL09Hinks.pdf",
    })

add_LS("while Loop Statements", GSCOPE_DICT)
LS[LS_idxs[u]-1].add_extension(timedelta(days=2))


date_ctx = MEETINGS[12]

add_CL("while Loops Practice + Positional Arguments", {
        "Sections 01 and 02": "/static/slides/alyssa/CL10.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL10Hinks.pdf",
    })

add_CQ("while Loops Practice", {"": "/cqs/while-loops.html"})


date_ctx = MEETINGS[13]

QZ[1].links = {
    "Practice Problems": "/resources/practice/practice-problems.html",
    "Review Session Recording": "https://youtu.be/z98ObWODtf4",
}

In [28]:
date_ctx = MEETINGS[14]

add_CL("Global Variables, Scope, and Importing", {
        "Sections 01 and 02": "/static/slides/alyssa/CL11.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL11Hinks.pdf",
    })

add_LS("Global Variables and Scope", GSCOPE_DICT)

add_LS("Importing Modules", GSCOPE_DICT)
LS[LS_idxs[u]-1].add_extension(timedelta(days=2))

add_EX("Wordle", {"": "/exercises/wordle-pt2.html"}, timedelta(days=8))

date_ctx = MEETINGS[15]

add_CL("Variables, Scope, and Importing Functions", {"": "/"})

add_CQ("Variables, Scope, and Importing Functions", {"": "/cqs/imports.html"})

CQ[CQ_idxs[u]-1].add_extension(timedelta(days=3))


date_ctx = MEETINGS[16]

add_CL("Lists", {
        "Sections 01 and 02": "/static/slides/alyssa/CL13.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL13Hinks.pdf",
    })

add_LS("Lists", GSCOPE_DICT)


date_ctx = MEETINGS[17]

add_CL("Lists in Memory", {
        "Sections 01 and 02": "/static/slides/alyssa/CL14.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL14Hinks_annotated.pdf",
    })

add_LS("Lists in Memory", GSCOPE_DICT)

date_ctx = MEETINGS[18]

add_CQ("Lists Practice", {"": "/cqs/lists.html"})

add_EX("List Utils", {"": "/exercises/list-utils.html"}, timedelta(days=9))

date_ctx = MEETINGS[19]

add_CL(
    "for Loops + for Loops in Memory",
    {
        "Sections 01 and 02": "/static/slides/alyssa/CL15.pdf",
        "Sections 03 and 04": "/static/slides/izzi/CL15Hinks.pdf",
    },
)

add_LS("for...in Loops", GSCOPE_DICT)

date_ctx = MEETINGS[20]

add_CQ("for Loops Practice", {"": "/cqs/for-loops.html"})

date_ctx = MEETINGS[21]

add_CL("Unit Tests", {"": "/"})

add_EX("List Unit Tests", {"": "/"}, timedelta(days=8))

date_ctx = MEETINGS[22]

add_CQ("Unit Tests", {"": "/cqs/list-unit-tests.html"})

date_ctx = MEETINGS[23]

add_CL("Dictionaries + Dictionaries in Memory", {"": "/"})

date_ctx = MEETINGS[24]

add_CQ("Dictionaries Practice", {"": "/cqs/dicts-for-loops.html"})

add_EX("Dictionary Utils", {"": "/"})

date_ctx = MEETINGS[25]


QZ[2].links = {
    "Practice Problems": "/resources/practice/practice-problems.html"
    }

In [29]:
date_ctx = MEETINGS[26]

add_CL("Object Oriented Programming", {"": "/"})

date_ctx = MEETINGS[27]

add_CL("Classes In Memory", {"": "/"})

date_ctx = MEETINGS[28]

add_CQ("OOP Practice", {"": "/cqs/oop-mutable.html"})

add_EX("River Simulation", {"": "/"}, timedelta(days=9))

date_ctx = MEETINGS[29]

add_CL("Magic Methods + Magic Methods in Memory", {"": "/"})

date_ctx = MEETINGS[30]

add_CQ("for Loops Practice", {"": "/cqs/magic_methods_etc.html"})

date_ctx = MEETINGS[31]



QZ[3].links = {
    "Practice Problems": "/resources/practice/practice-problems.html"
    }


In [30]:
date_ctx = MEETINGS[32]

add_CL("Reading Data from a CSV", {"": "/"})

add_EX("Data Utils", {"": "/"})

date_ctx = MEETINGS[33]

add_CL("Recursion", {"": "/"})

date_ctx = MEETINGS[34]

add_CQ("Recursion Practice: Reverse Engineering Recursive Functions",  {"": "/cqs/recursive-fns.html"})

date_ctx = MEETINGS[35]

add_CL("Algorithms + Runtime Analysis", {"": "/"})

add_EX("Recursive Turtle", {"": "/"})

date_ctx = MEETINGS[36]

add_CQ("Sort Algorithms",  {"": "/cqs/sort-alg.html"})


date_ctx = MEETINGS[37]



QZ[4].links = {
    "Practice Problems": "/resources/practice/practice-problems.html"
    }

In [31]:
date_ctx = MEETINGS[38]

add_CL("Recursive Structures", {"": "/"})

add_EX("Linked List Utils", {"": "/"}, timedelta(days=9))

date_ctx = MEETINGS[39]

add_CQ("Recursive Structures Practice", {"": "/cqs/recursion.html"})



date_ctx = MEETINGS[40]

add_CL("LDOC!", {"": "/"})



FN[0].links = {
    "Practice Problems": "/resources/practice/practice-problems.html"
    }

In [32]:
# LS_VIDEOS.extend(lessons)

#generate_VL_index_page(LS_VIDEOS)

In [ ]:

syllabus = merge(syllabus, quizzes)
for u in unit:
    syllabus = merge(syllabus, u)
serializable_syllabus = serialize(syllabus)
show_calendar(syllabus)

In [34]:
syllabus_copy = syllabus.copy()